# Incremental Ingestion with Timestamp (CDC Pattern)

This notebook demonstrates CDC-style incremental ingestion from CSV files using a **timestamp-based watermark** approach.

## How It Works

1. **Read** the last processed watermark timestamp from metadata
2. **Load** source CSV files
3. **Filter** records newer than the watermark
4. **Write** new/changed records to destination
5. **Update** the watermark with the latest timestamp

## Prerequisites

- CSV files with a `last_updated_ts` column uploaded to `/data/source/`
- Storage account configured with Synapse workspace

## Configuration

Update the storage account name below to match your deployment.

In [ ]:
# ============================================================================
# CONFIGURATION - Update these values for your environment
# ============================================================================

# Storage account name (from deployment output)
STORAGE_ACCOUNT = "<your-storage-account-name>"

# Container name
CONTAINER = "data"

# Paths
BASE_PATH = f"abfss://{CONTAINER}@{STORAGE_ACCOUNT}.dfs.core.windows.net"
SOURCE_PATH = f"{BASE_PATH}/source/"
DESTINATION_PATH = f"{BASE_PATH}/destination/with_timestamp/"
METADATA_PATH = f"{BASE_PATH}/metadata/"

# Watermark file
WATERMARK_FILE = f"{METADATA_PATH}watermark_timestamp.json"

# Source file pattern (files with timestamp column)
SOURCE_FILE_PATTERN = "customers_with_ts*.csv"

# Timestamp column name in source data
TIMESTAMP_COLUMN = "last_updated_ts"

# Primary key column for identifying records
PRIMARY_KEY_COLUMN = "customer_id"

print(f"Source Path: {SOURCE_PATH}")
print(f"Destination Path: {DESTINATION_PATH}")
print(f"Metadata Path: {METADATA_PATH}")

## Step 1: Read Current Watermark

The watermark stores the last processed timestamp. On first run, it will be initialized to a minimum date.

In [ ]:
import json
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max as spark_max, lit, to_timestamp

# Initialize Spark session (already available in Synapse)
spark = SparkSession.builder.getOrCreate()

def read_watermark():
    """
    Read the current watermark timestamp from metadata.
    Returns minimum date if watermark doesn't exist (first run).
    """
    try:
        # Try to read existing watermark
        watermark_df = spark.read.json(WATERMARK_FILE)
        watermark_value = watermark_df.select("last_watermark").collect()[0][0]
        print(f"Existing watermark found: {watermark_value}")
        return watermark_value
    except Exception as e:
        # First run - no watermark exists
        default_watermark = "1900-01-01 00:00:00"
        print(f"No existing watermark found. Using default: {default_watermark}")
        return default_watermark

# Read current watermark
current_watermark = read_watermark()
print(f"\nCurrent Watermark: {current_watermark}")

## Step 2: Load Source Data

Read all CSV files from the source folder that match our pattern.

In [ ]:
# Load source CSV files
source_file_path = f"{SOURCE_PATH}{SOURCE_FILE_PATTERN}"
print(f"Loading source files from: {source_file_path}")

try:
    # Read CSV files with header
    source_df = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv(source_file_path)
    
    # Convert timestamp column to proper timestamp type
    source_df = source_df.withColumn(
        TIMESTAMP_COLUMN, 
        to_timestamp(col(TIMESTAMP_COLUMN), "yyyy-MM-dd HH:mm:ss")
    )
    
    total_records = source_df.count()
    print(f"\nTotal records in source: {total_records}")
    print(f"\nSource Schema:")
    source_df.printSchema()
    print(f"\nSample source data:")
    source_df.show(5, truncate=False)
    
except Exception as e:
    print(f"Error loading source files: {str(e)}")
    print("Make sure CSV files are uploaded to the source folder.")
    raise

## Step 3: Filter Incremental Records

Select only records where the timestamp is greater than the current watermark.

In [ ]:
# Filter records newer than the watermark
incremental_df = source_df.filter(
    col(TIMESTAMP_COLUMN) > to_timestamp(lit(current_watermark), "yyyy-MM-dd HH:mm:ss")
)

incremental_count = incremental_df.count()
print(f"Records newer than watermark ({current_watermark}): {incremental_count}")

if incremental_count > 0:
    print(f"\nIncremental records to process:")
    incremental_df.orderBy(col(TIMESTAMP_COLUMN)).show(truncate=False)
else:
    print("\nNo new records to process. Data is up to date.")

## Step 4: Write Incremental Data to Destination

Append the new/changed records to the destination folder.

In [ ]:
if incremental_count > 0:
    # Add processing metadata
    from pyspark.sql.functions import current_timestamp
    
    output_df = incremental_df.withColumn("_processed_at", current_timestamp())
    
    # Write to destination (append mode)
    output_path = f"{DESTINATION_PATH}"
    print(f"Writing {incremental_count} records to: {output_path}")
    
    output_df.write \
        .mode("append") \
        .option("header", "true") \
        .csv(output_path)
    
    print(f"\nSuccessfully wrote {incremental_count} records to destination.")
else:
    print("No records to write. Skipping write operation.")

## Step 5: Update Watermark

Save the new watermark (maximum timestamp from processed records).

In [ ]:
def update_watermark(df, timestamp_column):
    """
    Update the watermark with the maximum timestamp from the processed data.
    """
    # Get the maximum timestamp from the data
    max_timestamp = df.agg(spark_max(col(timestamp_column))).collect()[0][0]
    
    if max_timestamp:
        # Format timestamp as string
        new_watermark = max_timestamp.strftime("%Y-%m-%d %H:%M:%S")
        
        # Create watermark record
        watermark_data = [{
            "last_watermark": new_watermark,
            "updated_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "records_processed": df.count()
        }]
        
        # Write watermark to metadata
        watermark_df = spark.createDataFrame(watermark_data)
        watermark_df.write.mode("overwrite").json(WATERMARK_FILE)
        
        print(f"Watermark updated to: {new_watermark}")
        return new_watermark
    else:
        print("No timestamp found. Watermark not updated.")
        return None

# Update watermark if we processed records
if incremental_count > 0:
    new_watermark = update_watermark(incremental_df, TIMESTAMP_COLUMN)
else:
    print("No records processed. Watermark remains unchanged.")

## Step 6: Verify Results

Check the destination folder and current watermark state.

In [ ]:
# Read and display destination data
print("=" * 60)
print("VERIFICATION: Current State")
print("=" * 60)

try:
    dest_df = spark.read \
        .option("header", "true") \
        .csv(DESTINATION_PATH)
    
    dest_count = dest_df.count()
    print(f"\nTotal records in destination: {dest_count}")
    print(f"\nDestination data sample:")
    dest_df.show(10, truncate=False)
except Exception as e:
    print(f"No data in destination yet or error reading: {str(e)}")

# Read current watermark
print(f"\n{'=' * 60}")
print("Current Watermark State:")
print("=" * 60)
try:
    watermark_df = spark.read.json(WATERMARK_FILE)
    watermark_df.show(truncate=False)
except Exception as e:
    print(f"No watermark file found: {str(e)}")

## Summary

This notebook demonstrated:

1. **Watermark-based CDC**: Using a timestamp column to track processed records
2. **Incremental Processing**: Only new/changed records are processed
3. **State Persistence**: Watermark stored in metadata for subsequent runs

### To Test Incremental Behavior:

1. Run this notebook with the initial dataset
2. Upload a new CSV file with records having newer timestamps
3. Run the notebook again - only new records will be processed

### Key Points:

- The watermark is stored as a JSON file in `/data/metadata/`
- Each run appends new data to `/data/destination/with_timestamp/`
- The `_processed_at` column tracks when records were ingested